# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Jan 27th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [ ]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd

In [ ]:
# We use from now on and forever the NEW bvec
# that were created by the EDDY command
# because they contain the small rotation
# that were introduced by motion correction
BVEC=$SUBJECTDIR'preprocessing/dwi_eddy.eddy_rotated_bvecs';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';

### QC of eddy

#### option 1: FSL eddyqc tool

The tool eddy_quad from FSL produce a pdf report froma single subject.
https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddyqc/UsersGuide

It is only available from FSL 6.0 and onward (for instance at the MPI, I have to use this environement "FSL --version 6.0.1")


eddy_quad <eddyBase> -idx <eddyIndex> -par <eddyParams> -m <mask> -b <bvals>
It takes as input the '--out' param from the eddy command (eddyBase).
The rest are all the other eddy command input; masks, bval, eddy index (indices) and eddy param (config)
    
#### If we had the data for topup, we would also need to include it here in the '-topup' flag

In [ ]:
eddy_quad dwi_eddy \
    -idx eddy_ind.txt \
    -par eddy_config.txt \
    -m mask_for_eddy.nii.gz \
    -b bvals.txt \
    -v

It creates a folder called dwi_eddy.qc containing a few things. We only care about the pdf.

The pdf contains graph of subject motion in translation and rotation through time.
Some graphs of eddy estimated outlier slices (remember that we are NOT using that information here, its just showing what eddy calculated)
And finaly we have some mean images post eddy.

In [ ]:
cd root
for SUBID in {01..21}; do echo 'sub_'$SUBID'/preprocessing/dwi_eddy.qc' >> myquads.txt; done
eddy_squad myquads.txt

#### option 2: Another quick DTI fit

In [ ]:
# Like we did before eddy, we do a quick dti fit and FA map
# using the eddy corrected data and the new bvec
dwi2tensor $SUBJECTDIR'preprocessing/dwi_eddy.nii.gz' $SUBJECTDIR'preprocessing/dti_post_eddy.nii.gz' -fslgrad $BVEC $BVAL;
tensor2metric $SUBJECTDIR'preprocessing/dti_post_eddy.nii.gz' -fa $SUBJECTDIR'preprocessing/fa_post_eddy.nii.gz';

# we can then visually compare it to the quick dti pre eddy.

In [ ]:
eddyqc quad

In [ ]:
dti post eddy, like before

In [ ]:
bet mask like before but now with erode
show atlas to justify, maybe prove the problem if you dont

In [ ]:
quicksyn fa to atlas
qc forward and inverse

In [ ]:
show and explain true command ants